In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.LazyLinear(128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

model = SimpleCNN().to(device)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train for few epochs
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/5], Loss: {running_loss/len(trainloader): .4f}')

torch.save(model.state_dict(), "simple_mnist_cnn.pth")


Activation maximization with Gradient Ascent

In [3]:
# Part 2: Activation Maximization

# Load model again in eval mode
model = SimpleCNN().to(device)
model.load_state_dict(torch.load("simple_mnist_cnn.pth"))
model.eval()

# Target class neuron — let's say 7
target_class = 7

# Start with random noise image
input_img = torch.randn((1, 1, 28, 28), requires_grad=True, device=device)

# Optimizer on input image
optimizer = optim.Adam([input_img], lr=0.1)

# Store images for visualization
images_over_time = []

# Activation Maximization loop
for step in range(200):
    optimizer.zero_grad()
    output = model(input_img)
    target_activation = output[0, target_class]
    
    # We want to maximize activation → loss = -activation
    loss = -target_activation
    loss.backward()
    optimizer.step()
    
    # Optional: clamp image pixels between 0 and 1
    input_img.data.clamp_(0, 1)
    
    # Store images at intervals
    if step % 20 == 0:
        img_cpu = input_img.detach().cpu().squeeze().clone()
        images_over_time.append(img_cpu)

print("Activation Maximization completed!")


NameError: name 'SimpleCNN' is not defined